# Tic Tac Toe Coin

To store state of tic tac toe game on Chia blockchain, we create an outer puzzle that wrap the inner tic tac toe puzzle. The coin puzzle outputs the conditions based on the output from the inner tic tac toe puzzle.

## Code
### [coin.clsp](./code/coin.clsp)
```lisp
;   MOD                         : the puzzle itself
;   PLAYER_ONE_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the first player
;   PLAYER_TWO_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the second player
;   CURRIED_TIC_TAC_TOE_PUZZLE  : tic tac toe puzzle with curried-in current board and next player
;   AMOUNT                      : coin amount (odd to be used with singleton top layer)
;   position                    : next play position as inner solution for curried tic tac toe puzzle

(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
     ...
)     
```

## Generate PKs for player one and two
```sh
cdv inspect keys --random | grep public -i
cdv inspect keys --random | grep public -i
Public Key: ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0
Public Key: 9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc
```


In [1]:
%%bash
chia version
cdv --version
python --version

1.5.0
cdv, version 1.0.8
Python 3.8.10


In [2]:
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)

In [3]:
sys.path.insert(0, "./code")
import tic_tac_toe

In [4]:
tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])
coin_puzzle = load_program("./code/coin.clsp", ["./code", "../../../shared"])

In [5]:
player_one_pk = bytes.fromhex("ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0")
player_one_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(player_one_pk).get_tree_hash()
player_two_pk = bytes.fromhex("9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc")
player_two_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(player_two_pk).get_tree_hash()
player_one_info = Program.to([player_one_pk, player_one_hash])
player_two_info = Program.to([player_two_pk, player_two_hash])

In [6]:
# empty board
board = [' '] * 9

# first player
player = 'x'

# (mod (BOARD V pos)
curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
        Program.to(board), 
        Program.to(player)
    ) 

#(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
curried_coin_puzzle = coin_puzzle.curry(
    coin_puzzle,
    player_one_info,
    player_two_info,
    curried_tic_tac_toe_puzzle)

In [7]:
amount = 1023
position = 4
print(f'{board} {player} {position}')
solution = Program.to([amount, position])
conditions = curried_coin_puzzle.run(solution)
print_program(conditions)
coin_puzzle_hash = conditions.at("rrfrf")

## verify puzzle
next_player = 'o'
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
        Program.to(board), 
        Program.to(next_player)
    )
curried_coin_puzzle = coin_puzzle.curry(
    coin_puzzle,
    player_one_info,
    player_two_info,
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '] x 4
((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0x24644d734eb6f3bebcc3314494518cf5df9fb8e2614981b45bc753912c85ab19) (73 1023) (51 0x25a404557dafb3b77eb6bf85a9717c55751c8cfed526392c43320755eb5ead7f 1023))
   |   |   
---+---+---
   | x |   
---+---+---
   |   |   

